In [ ]:
%%writefile transformer.py
import torch.nn as nn
from vit_pytorch import ViT

# Define the Vision Transformer model class
class TumorClassifierViT(nn.Module):
    def __init__(self, num_classes):
        super(TumorClassifierViT, self).__init__()
        self.vit = ViT(
            image_size = 224,
            patch_size = 32,
            num_classes = num_classes,
            dim = 1024,
            depth = 6,
            heads = 16,
            mlp_dim = 2048,
            dropout = 0.1,
            emb_dropout = 0.1
        )

    def forward(self, x):
        return self.vit(x)


Writing transformer.py


In [ ]:
from google.colab import files
files.upload()  # This will open a file upload dialog to select and upload the kaggle.json file.


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shashankdengi","key":"06fe8ea29e548b75cf41c4837b902a24"}'}

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
os.rename('kaggle.json', '/root/.kaggle/kaggle.json')


In [ ]:
!pip install kaggle


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset
!unzip brain-tumor-mri-dataset.zip -d ./data


Streaming output truncated to the last 5000 lines.
  inflating: ./data/Training/glioma/Tr-gl_0712.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0713.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0714.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0715.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0716.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0717.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0718.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0719.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0720.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0721.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0722.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0723.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0724.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0725.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0726.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0727.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0728.jpg  
  inflating: ./data/Training/glioma/Tr-gl_0729.j

In [ ]:
import os

# List files in the 'data' directory
print(os.listdir('./data'))


['Testing', 'Training']


In [ ]:
# Updated paths
train_dir = './data/Training'
test_dir = './data/Testing'


In [ ]:
# Check the structure inside the 'Training' directory
print("Training directory structure:")
print(os.listdir(train_dir))

# Check the structure inside the 'Testing' directory
print("\nTesting directory structure:")
print(os.listdir(test_dir))


Training directory structure:
['meningioma', 'glioma', 'notumor', 'pituitary']

Testing directory structure:
['meningioma', 'glioma', 'notumor', 'pituitary']


In [ ]:
!pip install vit-pytorch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [ ]:
!pip install arrow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.9 MB/s eta 0:00:00


In [ ]:
%%writefile train.py
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
from vit_pytorch import ViT
from vit_pytorch.vit import Transformer
from tqdm import tqdm
from transformer import TumorClassifierViT
from sklearn.metrics import confusion_matrix
import seaborn as sns
from prettytable import PrettyTable
import arrow

tableData = PrettyTable(['Epoch', 'Time Elapsed (HH:mm:ss)', 'Training Loss', 'Training Accuracy', 'Validation Loss', 'Validation Accuracy'])

# Set the device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data preprocessing and augmentation
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder('./data/Training', transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = ImageFolder('./data/Testing', transform=data_transforms)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load a batch of images and labels for visualization
data_iter = iter(train_loader)
images, labels = next(data_iter)

# Convert images to numpy arrays and denormalize
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
images = (images.numpy().transpose((0, 2, 3, 1)) * std + mean).clip(0, 1)

# Create a grid of images
num_images = len(images)
rows = int(np.ceil(num_images / 4))
fig, axes = plt.subplots(rows, 4, figsize=(15, 15))

# Plot images with labels
for i, ax in enumerate(axes.flat):
    if i < num_images:
        ax.imshow(images[i])
        ax.set_title(f'Label: {train_dataset.classes[labels[i]]}')
    ax.axis('off')

plt.savefig('classes.png')
plt.close()



model = TumorClassifierViT(num_classes=4).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Initialize lists to store training history
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Training loop
num_epochs = 100
best_val_accuracy = 0.0

startTime = arrow.now().timestamp()

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    all_labels = []
    all_predictions = []


    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    val_loss /= len(val_loader)
    val_accuracy = correct / total
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], ' f'Time elapsed: {arrow.get(arrow.now().timestamp() - startTime).format("HH:mm:ss")}'
           f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2%}, '
           f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}')

    #  'Epoch', 'Time Elapsed', 'Training Loss', 'Training Accuracy', 'Validation Loss', 'Validation Accuracy'
    currentEpoch = epoch + 1
    if currentEpoch % 5 == 0 or currentEpoch == 1:
        tableData.add_row([f'{currentEpoch}', f'{arrow.get(arrow.now().timestamp() - startTime).format("HH:mm:ss")}', f'{train_loss:.4f}', f'{train_accuracy:.4f}', f'{val_loss:.4f}' , f'{val_accuracy:.2%}'])

    # Save the best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model.pth')

accuracy = correct / total
print(f'Validation Accuracy: {accuracy:.2%}')

print(tableData)

# Visualize training history
plt.figure(figsize=(10, 7))
plt.subplot()
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss History')
plt.tight_layout()
plt.savefig('loss.png')
plt.close()

plt.figure(figsize=(10, 7))
plt.subplot()
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy History')
plt.tight_layout()
plt.savefig('accuracy.png')
plt.close()

# Visualize confusion matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=val_dataset.classes, yticklabels=val_dataset.classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.close()

Writing train.py


In [ ]:
!python train.py


Epoch 1/100: 100% 179/179 [00:54<00:00,  3.26it/s]
Epoch [1/100], Time elapsed: 00:01:02Training Loss: 377.9773, Training Accuracy: 32.58%, Validation Loss: 1.2886, Validation Accuracy: 39.36%
Epoch 2/100: 100% 179/179 [00:52<00:00,  3.41it/s]
Epoch [2/100], Time elapsed: 00:02:03Training Loss: 277.0270, Training Accuracy: 38.73%, Validation Loss: 1.3197, Validation Accuracy: 45.54%
Epoch 3/100: 100% 179/179 [00:53<00:00,  3.37it/s]
Epoch [3/100], Time elapsed: 00:03:04Training Loss: 253.8638, Training Accuracy: 42.70%, Validation Loss: 1.8743, Validation Accuracy: 37.99%
Epoch 4/100: 100% 179/179 [00:52<00:00,  3.42it/s]
Epoch [4/100], Time elapsed: 00:04:04Training Loss: 232.0836, Training Accuracy: 46.43%, Validation Loss: 2.2703, Validation Accuracy: 29.14%
Epoch 5/100: 100% 179/179 [00:52<00:00,  3.44it/s]
Epoch [5/100], Time elapsed: 00:05:04Training Loss: 173.7742, Training Accuracy: 59.44%, Validation Loss: 3.8188, Validation Accuracy: 31.43%
Epoch 6/100: 100% 179/179 [00:51<00

In [ ]:
from google.colab import files

# Download the best model (from the best fold)
files.download('best_model.pth')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformer import TumorClassifierViT
import torch


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
from google.colab import files
uploaded = files.upload()  # Select and upload best_model.pth from your local machine


Saving best_model.pth to best_model.pth


In [ ]:
from google.colab import files

# Upload files
uploaded = files.upload()

# Get the uploaded file names
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')
    # The file is stored in the current working directory (Colab environment)
    file_path = '/content/' + filename  # Path to the uploaded file
    print(f'File path: {file_path}')


Saving image(44).jpg to image(44).jpg
Uploaded file: image(44).jpg
File path: /content/image(44).jpg


In [ ]:
import os
print(os.listdir())  # This will show the list of files in the current directory


['.config', 'brain-tumor-mri-dataset.zip', '__pycache__', 'data', 'train.py', 'images.jpeg', 'best_model.pth', 'test.py', 'transformer.py', 'sample_data']


In [ ]:
%%writefile test.py
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from transformer import TumorClassifierViT

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define directories
train_dir = './data/Training'
test_dir = './data/Testing'  # Not used currently but can be added for future test cases

# Load the model
model = TumorClassifierViT(num_classes=4)
model.load_state_dict(torch.load('best_model.pth'))
model.to(device)
model.eval()

# Define the transformation
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the training dataset from the updated directory
train_dataset = ImageFolder(train_dir, transform=data_transforms)
class_names = train_dataset.classes

# Function to predict a single image
def predict_image(image_path, model, transform, device):
    # Open image
    image = Image.open(image_path).convert('RGB')
    # Apply transformations
    image_tensor = transform(image).unsqueeze(0)
    # Move to device
    image_tensor = image_tensor.to(device)

    # Predict
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

    confidence, predicted = torch.max(probabilities, 0)

    return predicted.item(), confidence.item(), probabilities.cpu().numpy(), image

# Path to the image (you can set it to any image from the test directory)
image_path = '/content/image(44).jpg'  # Updated to the newly uploaded image filename
  # Updated to the newly uploaded image filename


 # Replace this with the actual test image path if needed

# Predict the class
predicted_class, confidence, probabilities, image = predict_image(image_path, model, data_transforms, device)

# Print predicted class and confidence
print(f'Predicted class: {class_names[predicted_class]}')
print(f'Confidence: {confidence * 100}%')

# Create subplots: one for the image, one for the bar chart
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Display the image on the left subplot
axs[0].imshow(image)
axs[0].set_title(f'Predicted: {class_names[predicted_class]}')
axs[0].axis('off')

# Display the bar chart on the right subplot
print(probabilities)
multiply = lambda items: list(map(lambda x: x * 100, items))

rects = axs[1].barh(range(len(class_names)), multiply(probabilities),  align='center',
                     height=0.5, color='black')
axs[1].set_yticks(range(len(class_names)))
axs[1].set_yticklabels(class_names)
axs[1].set_xlim([0, 101])
axs[1].set_title('Confidence Level')
axs[1].xaxis.grid(True, linestyle='--', which='major',
                   color='grey', alpha=.25)

rect_labels = []
# Lastly, write in the ranking inside each bar to aid in interpretation
for rect in rects:
    width = int(rect.get_width())
    rankStr = f"{width}%"

    if width < 40:
        xloc = 5
        clr = 'black'
        align = 'left'
    else:
        xloc = -5
        clr = 'white'
        align = 'right'

    yloc = rect.get_y() + rect.get_height() / 2
    label = axs[1].annotate(rankStr, xy=(width, yloc), xytext=(xloc, 0),
                        textcoords="offset points",
                        ha=align, va='center',
                        color=clr, weight='bold', clip_on=True)
    rect_labels.append(label)

# Save the figure to disk
output_path = './prediction_result.png'
plt.savefig(output_path, bbox_inches='tight')
print(f'Figure saved to {output_path}')

# Show the figure
plt.show()

Overwriting test.py


In [ ]:
!python test.py


Predicted class: notumor
Confidence: 98.69471788406372%
[2.9866684e-05 1.3022876e-02 9.8694718e-01 6.3081075e-08]
Figure saved to ./prediction_result.png
Figure(1000x500)


In [ ]:
%%writefile test.py
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from transformer import TumorClassifierViT

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model
model = TumorClassifierViT(num_classes=4)
model.load_state_dict(torch.load('best_model.pth'))
model.to(device)
model.eval()

# Define the transformation
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to predict a single image
def predict_image(image_path, model, transform, device):
    # Open image
    image = Image.open(image_path).convert('RGB')
    # Apply transformations
    image_tensor = transform(image).unsqueeze(0)
    # Move to device
    image_tensor = image_tensor.to(device)

    # Predict
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

    confidence, predicted = torch.max(probabilities, 0)

    return predicted.item(), confidence.item(), probabilities.cpu().numpy(), image

# Path to the image
image_path = './content/images (1).jpeg'  # replace with your image path

# Predict the class
predicted_class, confidence, probabilities, image = predict_image(image_path, model, data_transforms, device)

# Define class names (assuming you know the class order)
train_dataset = ImageFolder('./data/Training', transform=data_transforms)
class_names = train_dataset.classes

# Print predicted class and confidence
print(f'Predicted class: {class_names[predicted_class]}')
print(f'Confidence: {confidence * 100}%')

# Create subplots: one for the image, one for the bar chart
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Display the image on the left subplot
axs[0].imshow(image)
axs[0].set_title(f'Predicted: {class_names[predicted_class]}')
axs[0].axis('off')

# Display the bar chart on the right subplot
print(probabilities)
multiply = lambda items: list(map(lambda x: x * 100, items))

rects = axs[1].barh(range(len(class_names)), multiply(probabilities),  align='center',
                     height=0.5, color='black')
axs[1].set_yticks(range(len(class_names)))
axs[1].set_yticklabels(class_names)
axs[1].set_xlim([0, 101])
axs[1].set_title('Confidence Level')
axs[1].xaxis.grid(True, linestyle='--', which='major',
                   color='grey', alpha=.25)

rect_labels = []
# Lastly, write in the ranking inside each bar to aid in interpretation
for rect in rects:
    # Rectangle widths are already integer-valued but are floating
    # type, so it helps to remove the trailing decimal point and 0 by
    # converting width to int type
    width = int(rect.get_width())

    rankStr = f"{width}%"
    # The bars aren't wide enough to print the ranking inside
    if width < 40:
        # Shift the text to the right side of the right edge
        xloc = 5
        # Black against white background
        clr = 'black'
        align = 'left'
    else:
        # Shift the text to the left side of the right edge
        xloc = -5
        # White on magenta
        clr = 'white'
        align = 'right'

    # Center the text vertically in the bar
    yloc = rect.get_y() + rect.get_height() / 2
    label = axs[1].annotate(rankStr, xy=(width, yloc), xytext=(xloc, 0),
                        textcoords="offset points",
                        ha=align, va='center',
                        color=clr, weight='bold', clip_on=True)
    rect_labels.append(label)

# Save the figure to disk
output_path = './prediction_result.png'
plt.savefig(output_path, bbox_inches='tight')
print(f'Figure saved to {output_path}')

# Show the figure
plt.show()

Overwriting test.py


In [ ]:
!python3 test.py


Traceback (most recent call last):
  File "/content/test.py", line 48, in <module>
    predicted_class, confidence, probabilities, image = predict_image(image_path, model, data_transforms, device)
                                                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/test.py", line 29, in predict_image
    image = Image.open(image_path).convert('RGB')
            ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/PIL/Image.py", line 3505, in open
    fp = builtins.open(filename, "rb")
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: './content/images (1).jpeg'
